In [ ]:
# import libraries
import netCDF4 as nc
import numpy as np
import glob as glob
import xarray as xr
import numpy.ma as ma
import matplotlib.pyplot as plt
from scipy.stats import percentileofscore
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# read the ncfile
f1obs = sorted(glob.glob(r'tmax.198*.nc'))
print(f1obs)
f2fut=sorted(glob.glob(r'tasmax_target*ssp*.nc'))
print(f2fut)
f3hismod=sorted(glob.glob(r'tasmax_target*historical*.nc'))
print(f3hismod)
# rad the ncfile variable
f1obsc = nc.MFDataset(f1obs)
# print(f1obsc)

# will cut data for 1980 to 1989, 3 years(due to memory size limit)
tmax1obs=f1obsc.variables['tmax']
print(tmax1obs)
tmax2obs=tmax1obs[0:3654,:,:]
print(tmax2obs)

print(tmax1obs)

print(len(tmax2obs))
print(tmax2obs.shape)

f2futc = nc.Dataset(f2fut[0])
print(f2futc)

# data for year 2050 as tasmax
tmax1fut=f2futc.variables['tasmax']
tmax1fut
tmax2fut=tmax1fut[:,:,:]-273.15
tmax2fut

print(tmax2fut.shape)

print(f3hismod[0])

f3hismodc1 = nc.Dataset(f3hismod[0])
print(f3hismodc1)
f3hismodc2 = nc.Dataset(f3hismod[1])
print(f3hismodc2)

tmax1hismod=f3hismodc1.variables['tasmax']
print(tmax1hismod)
tmax2hismod=f3hismodc2.variables['tasmax']
print(tmax2hismod)
tmax3hismod = np.concatenate((tmax1hismod, tmax2hismod), axis=0)
tmax3hismod=tmax3hismod-273.15
print(tmax3hismod)

print(tmax3hismod.shape)

tmax3hismodc=tmax3hismod[1827:2923,:,:]
print(tmax3hismodc.shape)

lon=f3hismodc1.variables['lon']
print(lon)
lat=f3hismodc1.variables['lat']
print(lat)

# define a function to get closest lat lon
def getclosest_ij(lats,lons,latpt,lonpt):
    # find squared distance of every point on grid
    lat_diff = np.abs(lats - latpt)
    lon_diff = np.abs(lons - lonpt)
    lat_idx = np.argmin(lat_diff)
    lon_idx = np.argmin(lon_diff)
    nearest_lat = latvals[lat_idx]
    nearest_lon = lonvals[lon_idx]
    # Get 2D index for latvals and lonvals arrays from 1D index
    return nearest_lat,nearest_lon, lat_idx, lon_idx
# enter the lat lon of a desired place to get temperature data for SSP585 for future year 2050
latvals = lat[:]; lonvals = lon[:]
latuser=22.31
lonuseradjust=73.53
if lonuseradjust<0:
    lonuser=360+lonuseradjust
else:
    lonuser=lonuseradjust
print("lat: {}, lon: {}".format(latuser, lonuser))

ans1 = np.empty((365,360,720))
# ini=1
# x2=1;
# y2=0;
# get outputfor i 135 and j 147
for i in range(0, 360):
    for j in range(0,720):
        latuser=lat[i]
        lonuser=lon[j]
        x1, y1, x2, y2 = getclosest_ij(latvals, lonvals, latuser, lonuser)
        # print(x1)
        # print(y1)
        print("latidx",x2)
        print("lonidx",y2)
        nearest_lat = latvals[x2]
        nearest_lon = lonvals[y2]
        # print("nearest_lat: {}, nearest_lon: {}".format(nearest_lat, nearest_lon))
        tmax2obs1=tmax2obs[:,x2,y2]
        tmax3hismodc1=tmax3hismodc[:,x2,y2]
        tmax2fut1=tmax2fut[:,x2,y2]
        
        tmax3obs = ma.filled(tmax2obs1, np.nan)
        tmax4hismodc = ma.filled(tmax3hismodc1, np.nan)
        tmax3fut = ma.filled(tmax2fut1, np.nan)
        
        # print(tmax3obs.shape)
        
        lon=f2futc.variables['lon']
        # print(lon)
        lat=f2futc.variables['lat']
        # print(lat)
        
        # print(lon[:])
        
        lon=f1obsc.variables['lon']
        # print(lon)
        lat=f1obsc.variables['lat']
        # print(lat)
        
        # print(lat[:])
        
        obs_nan_mask=tmax2obs.mask
        # obs_nan_mask.shape
        obs_nan_maskfut=obs_nan_mask[0:365]
        # obs_nan_maskfut.shape
        
        observed_flat = tmax3obs
        his_model_flat = tmax4hismodc
        fut_model_flat = tmax3fut
        
        # print(observed_flat.shape)
        
        observed_flatwonan=observed_flat[~np.isnan(observed_flat)]
        his_model_flatwonan=his_model_flat[~np.isnan(his_model_flat)]
        fut_model_flatwonan=fut_model_flat[~np.isnan(fut_model_flat)]
        # print(observed_flatwonan.shape)
        
        model_present_corrected = np.zeros(his_model_flat.size)  
        model_future_corrected = np.zeros(fut_model_flat.size)
        # model_present_corrected = np.zeros(his_model_flatwonan.size)  
        # model_future_corrected = np.zeros(fut_model_flatwonan.size)
        if np.all(np.isnan(observed_flat)):
                print(f"Skipping iteration {x2,y2} as the entire matrix is NaN.")
                ans1[:,i,j]=0;
        else:
            for ival, model_value in enumerate(his_model_flatwonan):
                percentile = percentileofscore(his_model_flatwonan, model_value)
                model_present_corrected[ival] = np.percentile(observed_flatwonan, percentile)
                # print("Done1")

            for ival, model_value in enumerate(fut_model_flatwonan):
                percentile = percentileofscore(fut_model_flatwonan, model_value)
                model_future_corrected[ival] = model_value + np.percentile(observed_flatwonan, percentile) - np.percentile(his_model_flatwonan, percentile)
                # print("Done2")

            ans1[:,i,j]=model_future_corrected;

            # Sort the data and plot cdf for not corrected datset
            sorted_data1 = np.sort(observed_flat)
            sorted_data2 = np.sort(his_model_flatwonan)
            sorted_data3 = np.sort(fut_model_flatwonan)
            # Compute the CDF
            cdf1 = np.arange(1, len(sorted_data1) + 1) / len(sorted_data1)
            cdf2 = np.arange(1, len(sorted_data2) + 1) / len(sorted_data2)
            cdf3 = np.arange(1, len(sorted_data3) + 1) / len(sorted_data3)

            # Plot the CDFs
    #         plt.figure(figsize=(8, 5))
    #         plt.plot(sorted_data1, cdf1, label='Dataset 1')
    #         plt.plot(sorted_data2, cdf2, label='Dataset 2')
    #         plt.plot(sorted_data3, cdf3, label='Dataset 3')


    #         plt.title('Cumulative Distribution Function (CDF) of Multiple Datasets')
    #         plt.xlabel('Data Values')
    #         plt.ylabel('Cumulative Probability')
    #         plt.legend()
    #         plt.grid(True)
    #         plt.show()

            # Sort the data and plot for corrected dataset
            sorted_data1 = np.sort(observed_flat)
            sorted_data2 = np.sort(model_present_corrected)
            sorted_data3 = np.sort(model_future_corrected)
            # Compute the CDF
            cdf1 = np.arange(1, len(sorted_data1) + 1) / len(sorted_data1)
            cdf2 = np.arange(1, len(sorted_data2) + 1) / len(sorted_data2)
            cdf3 = np.arange(1, len(sorted_data3) + 1) / len(sorted_data3)

            # Plot the CDFs
    #         plt.figure(figsize=(8, 5))
    #         plt.plot(sorted_data1, cdf1, label='Dataset 1')
    #         plt.plot(sorted_data2, cdf2, label='Dataset 2')
    #         plt.plot(sorted_data3, cdf3, label='Dataset 3')

    #         plt.title('Cumulative Distribution Function (CDF) of Multiple Datasets')
    #         plt.xlabel('Data Values')
    #         plt.ylabel('Cumulative Probability')
    #         plt.legend()
    #         plt.grid(True)
    #         plt.show()

            # print(sorted_data3.shape)

    #         plt.figure(figsize=(8, 5))
    #         days=np.arange(1, 366, 1)
    #         threshold_temperature=35
    #         plt.plot(days, model_future_corrected, label='Daily Temperature (°C)')
    #         # Add a horizontal line for the threshold temperature
    #         plt.axhline(y=threshold_temperature, color='r', linestyle='--', label=f'Threshold: {threshold_temperature}°C')

    #         plt.title('Daily maximum temperature for year 2050')
    #         plt.xlabel('Days')
    #         plt.ylabel('Temperature')
    #         plt.legend()
    #         plt.grid(True)
    #         plt.show()

            # Count the number of days above the threshold
            daily_temperatures=model_future_corrected
            days_above_threshold = np.sum(daily_temperatures > threshold_temperature)


            # Display the result
    #         print(f"Number of days above {threshold_temperature}°C: {days_above_threshold} days")
            # print("Done")
max_temp = np.max(ans1, axis=0)

# plot the data for whole world
lats = lat
lons = lon
grid_lats, grid_lons = np.meshgrid(lons, lats)
grid_temps=max_temp
# grid_temps=np.flip(max_temp,axis=0)
# grid_temps=np.flip(grid_temps1,axis=0)

# Plot the grid map of temperature
plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
plt.contourf(grid_lats, grid_lons, grid_temps, cmap='coolwarm')
plt.colorbar(label='Temperature (°C)')

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines()

# Customize the plot
plt.title('Grid Map of Temperature')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()